In [79]:
import cv2
import numpy as np
from pyzbar.pyzbar import decode
import os
import zxingcpp

qcd = cv2.QRCodeDetector()
# Create a QRCodeDetector object
qr_detector = cv2.QRCodeDetector()

In [83]:
# Get a list of all files in the folder
folder_path = "mainImageFile/"
image_files = [f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))]

In [89]:
# Initialize the minimum number
min_number = 99
list_Number_Zxing = []
list_Number_Pyzbar = []

# Loop over each file
for image_file in image_files:
    # Load the image
    image = cv2.imread(folder_path+'/'+image_file)

    # Convert the image to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Threshold the image
    ret, threshold = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

    decoded_data = decode(threshold)
    if decoded_data:
        # Loop over each QR code in the image
        for qrCode in decoded_data:
            # Get the number from the QR code
            number = int(qrCode.data)
            list_Number_Pyzbar.append(number)

            # Update the minimum number if necessary
            if number < min_number:
                min_number = number
                
    results = zxingcpp.read_barcodes(threshold)
    for result in results:
        number = int(result.text)
        list_Number_Zxing.append(number)
        if number < min_number:
                min_number = number

if( min_number == 99 ):
    print("Photos couldn't be read")
else:
    print("Minimum number from all QR codes:", min_number)
    print('Pyzbar:',list_Number_Pyzbar)
    print('Zxing',list_Number_Zxing)

Minimum number from all QR codes: 2
Pyzbar: [17, 11, 5, 3, 2, 6]
Zxing [17, 25, 11, 5, 7, 3, 2, 4, 6, 13, 29]


In [65]:
# Threshold the image
ret, threshold = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

cv2.imwrite("contour/threshold.jpg", gray)

# Find the contours of the QR codes
contours, hierarchy = cv2.findContours(threshold, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
len(contours)

decoded_data = decode(threshold)
if decoded_data:
    # cv2.imwrite("contour/contour_" + str(qrCount) + ".jpg", qr_code_image)
    # print(retval,decoded_info)
    for qrCode in decoded_data:
        print(qrCode.data)


In [66]:
qrCount = 0

for cnt in contours:
    # Check if the contour is convex
    if cv2.isContourConvex(cnt):
        
        # Get the bounding rectangle of the QR code
        x, y, w, h = cv2.boundingRect(cnt)
        # print(w,h)
        
        # Extract the QR code from the image
        qr_code_image = threshold[y:y+h, x:x+w]
        
        # qr_code_image = sharpen_image(qr_code_image)
        
        # Use a QR code library to decode the QR code and get the encoded information
        # retval, decoded_info, points, straight_qrcode = qcd.detectAndDecodeMulti(qr_code_image)
        # print(qr_code_image)
        # cv2.imwrite("contour/contour_" + str(qrCount) + ".jpg", qr_code_image)
        qrCount += 1
        
        if( w >= 20 and h >= 20 ):
            decoded_data = decode(qr_code_image)
            cv2.imwrite("contour/contour_" + str(qrCount) + ".jpg", qr_code_image)
        
            if decoded_data:
                # cv2.imwrite("contour/contour_" + str(qrCount) + ".jpg", qr_code_image)
                print(1)
                # print(retval,decoded_info)
                for qrCode in decoded_data:
                    print(qrCode.data)
            
            
print(qrCount)

        

171


In [ ]:
img = cv2.imread('contour/2.jpg')

# img = sharpen_image(img)
# cv2.imwrite("contour/contour_" + str(qrCount) + ".jpg", qr_code_image)

qcd = cv2.QRCodeDetector()
retval, decoded_info, points, straight_qrcode = qcd.detectAndDecodeMulti(img)

retval
decoded_info


In [ ]:
from pyzbar.pyzbar import decode
from PIL import Image

# Read the QR code from an image file
decoded_data = decode(img)

decoded_data
# Print the decoded QR code data
# print(decoded_data[0].data.decode("utf-8"))

In [ ]:
def crop_box(image, kp):
    # Get the coordinates of the keypoints
    coords = np.array([kp[i].pt for i in range(len(kp))])

    # Get the bounding rectangle of the keypoints
    x, y, w, h = cv2.boundingRect(coords)

    # Crop the image using the bounding rectangle coordinates
    crop_img = image[y:y+h, x:x+w]

    return crop_img

In [ ]:
def detect_boxes(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    sift = cv2.xfeatures2d.SIFT_create()
    kp, des = sift.detectAndCompute(gray, None)
    img = cv2.drawKeypoints(gray, kp, None)
    # cv2.imshow("Keypoints", img)
    # cv2.waitKey(0)
    # cv2.destroyAllWindows()
    print(len(kp))
    boxes = crop_box(img,kp)
    
detect_boxes(cv2.imread("TestImg/zz.png"))